# Intro dlt -> LanceDB loading

## Install requirements

To create a json -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model 
<a href="https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview">sentence-transformers/all-MiniLM-L6-v2</a>

In [1]:
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

zsh:1: no matches found: dlt[lancedb]==0.5.1a0
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 410.2 kB/s eta 0:00:001m436.3 kB/s eta 0:00:01
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 2.1 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 608.6 kB/s eta 0:00:00m eta 0:00:010:00:03
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 M

## Load the data

We'll first load the data just into LanceDB, without embedding it. LanceDB stores both the data and the embeddings, and can also embed data and queries on the fly.

Some definitions:
* A dlt **source** is a grouping of **resources** (e.g. all your data from Hubspot)
* A dlt **resource** is a function that yields data (e.g. a function yielding all your Hubspot companies)
* A dlt **pipeline** is how you ingest your data

Loading the data consists of a few steps:
1. Use the requests library to get the data
2. Define a dlt resource that yields the individual documents
3. Create a dlt pipeline and run it

In [12]:
import requests
import dlt
import os
from dlt.destinations.adapters import lancedb_adapter

In [7]:
qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

/Users/Aaron/anaconda3/envs/py312_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
documents
[{'name': 'text', 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': '

In [9]:
import lancedb
db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents']


In [10]:
db_table = db.open_table("qanda___documents")

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,2da55a48-f9c2-5212-b807-29d0ff23a3d6,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721141700.70987,XMR/HNq1mLbm5Q
1,89a9b796-d8c3-5cb6-98d6-769ae87308b9,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721141700.70987,GQig4sLK5gDGCg
2,c69a8fdc-b512-510a-8c2c-3881e32a6b3d,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721141700.70987,eIlgFythKCDITA
3,aec43c4c-4dac-524e-b6fe-01b6c57c60ba,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721141700.70987,U4tYyeNopoGztg
4,67d1fee7-9c7b-53ef-801b-eaad216e1fba,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721141700.70987,FV0FSrLd75cjvA
...,...,...,...,...,...,...
943,20a8de85-4496-56d9-86df-9ce333fc081e,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721141700.70987,sXIiR9Cuw6WQCQ
944,c3919fa6-8c72-55f4-bcd2-f428494b49a4,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721141700.70987,kHDeLJcs0+3u1w
945,31343233-bc09-57f1-822e-f3fd9a4a50d4,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721141700.70987,Kwn0NXFaRNJvcQ
946,1e75631d-b41f-5b01-b036-cf10b80f8192,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721141700.70987,IiNMgj++J1mqzQ


## Load and embed the data

Now we load the same data again (into a new table), but embed it directly with the `lancedb_adapter`. This consists of the following steps:

1. Define the embedding model to use via ENV variables
2. Define a new pipeline to load the same data and embed the "text" and "question" columns with the `lancedb_adapter`

You can use any embedding model, from open source to OpenAI. We've chosen the [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) sentence transformer for speed and simplicty.

Note: this pipeline runs slightly longer because it has to download the model and embed the data.

In [16]:
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"
pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")
load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")
print(load_info)

_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
documents
[{'name': 'text', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullabl

In [17]:
db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [18]:
db_table = db.open_table("qanda_embedded___documents")

db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,9b6b8e02-7473-529b-8b65-5b90a44c4ccc,"[-0.00035096044, -0.062014237, -0.03799987, 0....",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721153497.323018,xDljAqW4Go+btw
1,ff3d1217-a66e-5744-8e2e-ffb044b8cd12,"[0.02001144, -0.011535538, 0.013017191, -0.002...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721153497.323018,lXnoJuU12T4gXA
2,81109af6-2f99-5b3f-a3eb-b5fc208cd4da,"[0.01485756, -0.06664991, -0.013571257, 0.0232...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721153497.323018,Zs34+PWIxmxARw
3,d1faac1c-a142-5cef-a493-dc392c68bcfb,"[-0.023312058, -0.09461489, 0.05636162, -0.001...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721153497.323018,UkYTdZP1VbP8Kg
4,a4190429-38b8-5994-8a0f-a7cee5b293d0,"[0.026537674, -0.017796598, 0.0021156033, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721153497.323018,jSDNTO0+ADbT5g
...,...,...,...,...,...,...,...
943,7e5bcc2a-78ff-5b1e-908d-82db97e08b3a,"[0.016619347, -0.033603188, -0.09334718, -0.02...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721153497.323018,KIIPH7NoN0UumA
944,628b7201-412d-5ed7-a264-ae06b0d3a522,"[0.026872864, -0.0019948678, 0.008369055, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721153497.323018,W5Kx/1j92ul1EQ
945,ace2a7c9-53dc-5757-b306-52052eafdce9,"[0.035137653, 0.05626561, 0.02442844, -0.06512...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721153497.323018,xOZcqQ5QX7FAqQ
946,b0bca538-93ec-511d-a5fa-806aa7c84156,"[0.033809807, -0.003121895, 0.0017485361, 0.01...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721153497.323018,7K8G/12zvF0TQw


# Create an up-to-date RAG with dlt and LanceDB
In this demo, we will be creating an LLM chat bot that has the latest knowledge of the employee handbook of a fictional company. We will be able to chat to it about specific policies like PTO, work from home etc.

To build this, we would need to do three things:
1. The company policies exist in a [Notion Page](https://dlthub.notion.site/Employee-handbook-669c2a1e04044465811c8ca22977685d). We will need to first extract the text from these pages.
2. Once extracted, we will want to embed them into vectors and then store them in a vector database.
3. This will allow us to create our RAG: a function that would accept a user question, match it to the information stored in the vector database, and then send the question + relevant information as input to the LLM.

#### We will be using the following OSS tools for this:
1. dlt for data ingestion:  
     1. dlt can easily connect to any REST API source (like Notion)
     2. It also has integrations with vector databases, like LanceDB.
     3. It also allows to easily plug in functionality like incremental loading.
2. LanceDB as a vector database:
     1. LanceDB is an open-source vector database that is very easy to use and integrate into python workflows
     2. It is in-process and serverless (like DuckDB), which makes querying and retreival very efficient
3. Ollama for RAG:
     1. Ollama is open-source and allows you to easily run LLMs locally

## Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

To create a notion -> lancedb pipeline, we need to install:
  1. dlt with lancedb extras
  2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we     choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

### 2. Create a dlt project with rest_api source and lancedb destination
We now create a dlt project using the command `dlt init <source> <destination>`.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [19]:
!yes | dlt init rest_api lancedb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information
yes: stdout: Broken pipe


### 3. Add API credentials

To access APIs, databases, or any third-party applications, one might need to specify relevant credentials.

With dlt, we can do it in two ways:
1. Pass the credentials and any other sensitive information inside `.dlt/secrets.toml`
  ```toml
  [sources.rest_api.notion]
  api_key = "notion api key"

  [destination.lancedb]
  embedding_model_provider = "sentence-transformers"
  embedding_model = "all-MiniLM-L6-v2"

  [destination.lancedb.credentials]
  uri = ".lancedb"
  api_key = "api_key"
  embedding_model_provider_api_key = "embedding_model_provider_api_key"
  ```
2. Pass them as environment variables
  ```python
  import os
  
  os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "notion api key"

  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

  os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__API_KEY"] = "api_key"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__EMBEDDING_MODEL_PROVIDER_API_KEY"] = "embedding_model_provider_api_key"
  ```

We are going to be using option 2. It's not advisable to paste sensitive information like API keys inside the code, so instead we're going to include them inside the secrets tab in the side panel of the notebook. This will allow us to access the secret values from the notebook.

Since we are using the OSS version of LanceDB and OSS embedding models, we only need to specify the API key for Notion.

**Note**: You will need to copy the [notion API key](https://share.1password.com/s#ohRHKjRIGagH_7HzxHzieZViCefOUmodTs2vodixXdQ) into the secrets tab under the name `SOURCES__REST_API__NOTION__API_KEY`. Make sure to enable notebook access after pasting the key.

In [22]:
import os


os.environ["SOURCES__REST_API__NOTION__API_KEY"] = 's#ohRHKjRIGagH_7HzxHzieZViCefOUmodTs2vodixXdQ'

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

### 4. Write the pipeline code

Note: We first go over the code step by step before putting it into runnable cells

Import necessary modules (run this cell)

In [23]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

2. Configure the dlt rest api source to connect to and extract the relevant data out from the Notion REST API.

  Our notion space has multiple pages and each page has multiple paragraphs (called blocks). To extract all this data from the Notion API, we would first need to get a list of all the page_ids (each page has a unique page_id), and then use the page_id to request the contents from the individual pages. Specifically:
  1. We will first request the page_ids from the `/search` endpoint
  2. And then using the returned page_ids, we will request the contents from the `/blocks/{page_id}/children` endpoint

  With this in mind, we can configure the dlt notion rest api source as follows:

2. Configure the dlt rest api source to connect to and extract the relevant data out from the Notion REST API.

  Our notion space has multiple pages and each page has multiple paragraphs (called blocks). To extract all this data from the Notion API, we would first need to get a list of all the page_ids (each page has a unique page_id), and then use the page_id to request the contents from the individual pages. Specifically:
  1. We will first request the page_ids from the `/search` endpoint
  2. And then using the returned page_ids, we will request the contents from the `/blocks/{page_id}/children` endpoint

  With this in mind, we can configure the dlt notion rest api source as follows:
```python
  RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }
```

Explanation:
  1. `client`: Here we added our base url, headers, and authentication
  2. `resources`: This is a list of endpoints that we wish to request data from (here: `/search` and `/blocks/{page_id}/children`)
  3. [`/search`](https://developers.notion.com/reference/post-search) endpoint:
      - The Notion API search endpoint allows us to filter pages based on the title. We can specify which pages we want returned based on the parameter "query". For example, if we'd like to return only those pages which has the word "workshop" in the title, then we would set `"query": "workshop"` in the json body.    
      - As a response, it returns only page metadata (like page_id). Example response:


```json
    {
      "object": "list",
      "results": [
        {
          "object": "page",
          "id": "954b67f9-3f87-41db-8874-23b92bbd31ee",
          "created_time": "2022-07-06T19:30:00.000Z",
          "last_edited_time": "2022-07-06T19:30:00.000Z",
          .
          .
          .
      ],
      "next_cursor": null,
      "has_more": false,
      "type": "page_or_database",
      "page_or_database": {}
    }
```

 - This is how we would define our endpoint configuration for `/search`:
```python
      {
        "name": "search",
        "endpoint": {
            "path": "search",
            "method": "POST",
            "paginator": PostBodyPaginator(),
            "json": {
                "query": "workshop",
                "sort": {
                    "direction": "ascending",
                    "timestamp": "last_edited_time"
                }
            },
            "data_selector": "results"
        }
    },

- `paginator` allows us to specify the pagination strategy relevant for the API and the endpoint. (More on this later)
- Since `/search` is a POST endpoint, we can include the json body inside the key `json`.
- We don't need the whole JSON response, but only the contents inside the field "results". We filter this out by specifying `"data_selector": "results"`.

4. [`blocks/{page_id}/children`](https://developers.notion.com/reference/get-block-children) endpoint:
  - This is a GET point that returns a list of block objects (in our case, paragraphs) from a specific page.
  - Since it accepts page_id as a parameter, we can pass this inside the key `params`
  - We would like to be able to automatically fetch the page_ids returned from the `/search` endpoint and pass it as parameter into the endpoint `blocks/{page_id}/children`. We can do this by linking the two resources as follows:

```python
{
      "name": "page_content",
      "endpoint": {
          "path": "blocks/{page_id}/children",
          "paginator": JSONResponsePaginator(),
          "params": {
              "page_id": {
                  "type": "resolve",
                  "resource": "search",
                  "field": "id"
              }
          },
      }
}
```
- By specifying `"type":"resolve"`, we are letting dlt know that this parameter needs to be resolved from the parent resource `"search"` using the field `"id"`, which corresponds to the page id in the response of `/search`.